In [1]:
import numpy as np
from math import log, exp
from collections import Counter

In [2]:
#Datasets & Utilities

def make_regression_data(n=200, d=1, noise=1.0, random_state=0):
    rng = np.random.RandomState(random_state)
    X = rng.randn(n, d)
    w = rng.randn(d,)
    y = X.dot(w) + noise * rng.randn(n)
    return X, y, w

def make_classification_data(n=200, d=2, random_state=0):
    rng = np.random.RandomState(random_state)
    X0 = rng.randn(n//2, d) + np.array([-2.0]*d)
    X1 = rng.randn(n//2, d) + np.array([2.0]*d)
    X = np.vstack([X0, X1])
    y = np.array([0]*(n//2) + [1]*(n//2))
    perm = rng.permutation(n)
    return X[perm], y[perm]

In [3]:
#Supervised- Linear Regression

class LinearRegressionClosedForm:
    """
    Ordinary Least Squares (OLS) linear regression.
    Theory: Solve for w in closed-form: w = (X^T X + lambda I)^{-1} X^T y
    This implementation supports an L2 regularization parameter `lam`.
    """
    def __init__(self, fit_intercept=True, lam=0.0):
        self.fit_intercept = fit_intercept
        self.lam = lam
        self.coef_ = None
        self.intercept_ = 0.0

    def _add_intercept(self, X):
        if self.fit_intercept:
            return np.hstack([np.ones((X.shape[0], 1)), X])
        return X

    def fit(self, X, y):
        X0 = self._add_intercept(X)
        n, p = X0.shape
        A = X0.T.dot(X0)
        A += self.lam * np.eye(p)
        w = np.linalg.solve(A, X0.T.dot(y))
        if self.fit_intercept:
            self.intercept_ = w[0]
            self.coef_ = w[1:]
        else:
            self.intercept_ = 0.0
            self.coef_ = w
        return self

    def predict(self, X):
        return X.dot(self.coef_) + self.intercept_

class LinearRegressionGD:
    """
    Linear regression using gradient descent (MSE loss).
    """
    def __init__(self, lr=1e-2, n_iters=1000, fit_intercept=True, lam=0.0):
        self.lr = lr
        self.n_iters = n_iters
        self.fit_intercept = fit_intercept
        self.lam = lam

    def fit(self, X, y):
        X0 = np.hstack([np.ones((X.shape[0], 1)), X]) if self.fit_intercept else X
        n, p = X0.shape
        w = np.zeros(p)
        for i in range(self.n_iters):
            preds = X0.dot(w)
            grad = (2.0/n) * X0.T.dot(preds - y) + 2*self.lam*w
            w -= self.lr * grad
        if self.fit_intercept:
            self.intercept_ = w[0]
            self.coef_ = w[1:]
        else:
            self.intercept_ = 0.0
            self.coef_ = w
        return self

    def predict(self, X):
        return X.dot(self.coef_) + self.intercept_

In [4]:
#Supervised: Logistic Regression

class LogisticRegressionGD:
    """
    Binary logistic regression trained with batch gradient descent.
    Uses sigmoid activation and cross-entropy loss.
    Supports L2 regularization.
    """
    def __init__(self, lr=1e-2, n_iters=1000, fit_intercept=True, lam=0.0):
        self.lr = lr
        self.n_iters = n_iters
        self.fit_intercept = fit_intercept
        self.lam = lam

    def _sigmoid(self, z):
        return 1.0 / (1.0 + np.exp(-z))

    def fit(self, X, y):
        X0 = np.hstack([np.ones((X.shape[0], 1)), X]) if self.fit_intercept else X
        n, p = X0.shape
        w = np.zeros(p)
        for i in range(self.n_iters):
            z = X0.dot(w)
            preds = self._sigmoid(z)
            grad = (1.0/n) * X0.T.dot(preds - y) + 2*self.lam*w
            w -= self.lr * grad
        if self.fit_intercept:
            self.intercept_ = w[0]
            self.coef_ = w[1:]
        else:
            self.intercept_ = 0.0
            self.coef_ = w
        return self

    def predict_proba(self, X):
        z = X.dot(self.coef_) + self.intercept_
        p = self._sigmoid(z)
        return np.vstack([1-p, p]).T

    def predict(self, X, threshold=0.5):
        return (self.predict_proba(X)[:,1] >= threshold).astype(int)

In [5]:
#Supervised: k-Nearest Neighbor

class KNNClassifier:
    """
    Simple k-Nearest Neighbors classifier using brute-force distance.
    """
    def __init__(self, k=5):
        self.k = k

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        return self

    def predict(self, X):
        preds = []
        for x in X:
            dists = np.linalg.norm(self.X_train - x, axis=1)
            idx = np.argsort(dists)[:self.k]
            vals = self.y_train[idx]
            preds.append(Counter(vals).most_common(1)[0][0])
        return np.array(preds)

In [6]:
# Supervised: Decision Tree (classification)

class DecisionTreeClassifierSimple:
    """
    A compact recursive CART-like decision tree for classification.
    Uses Gini impurity and axis-aligned splits.
    Not optimized for performance; educational implementation.
    """

    class Node:
        def __init__(self, gini, n_samples, predicted_class):
            self.gini = gini
            self.n_samples = n_samples
            self.predicted_class = predicted_class
            self.feature_index = None
            self.threshold = None
            self.left = None
            self.right = None

    def __init__(self, max_depth=5, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.n_classes_ = None
        self.n_features_ = None
        self.root = None

    def _gini(self, y):
        m = len(y)
        if m == 0:
            return 0
        counts = np.bincount(y)
        probs = counts / m
        return 1.0 - np.sum(probs**2)

    def _best_split(self, X, y):
        m, n = X.shape
        if m < 2:
            return None, None
        best_gini = 1.0
        best_idx, best_thr = None, None
        current_gini = self._gini(y)
        for idx in range(n):
            thresholds = np.unique(X[:, idx])
            for thr in thresholds:
                left_mask = X[:, idx] <= thr
                y_left, y_right = y[left_mask], y[~left_mask]
                if len(y_left) == 0 or len(y_right) == 0:
                    continue
                g_left = self._gini(y_left)
                g_right = self._gini(y_right)
                g = (len(y_left) * g_left + len(y_right) * g_right) / m
                if g < best_gini:
                    best_gini = g
                    best_idx = idx
                    best_thr = thr
        return best_idx, best_thr

    def _build(self, X, y, depth=0):
        num_samples_per_class = [np.sum(y == i) for i in range(np.max(y)+1)]
        predicted_class = np.argmax(num_samples_per_class)
        node = self.Node(gini=self._gini(y), n_samples=len(y), predicted_class=predicted_class)
        if depth < self.max_depth and len(y) >= self.min_samples_split and node.gini > 0.0:
            idx, thr = self._best_split(X, y)
            if idx is not None:
                left_mask = X[:, idx] <= thr
                node.feature_index = idx
                node.threshold = thr
                node.left = self._build(X[left_mask], y[left_mask], depth+1)
                node.right = self._build(X[~left_mask], y[~left_mask], depth+1)
        return node

    def fit(self, X, y):
        self.n_classes_ = len(np.unique(y))
        self.n_features_ = X.shape[1]
        self.root = self._build(X, y)
        return self

    def _predict_one(self, inputs, node):
        while node.left:
            if inputs[node.feature_index] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node.predicted_class

    def predict(self, X):
        return np.array([self._predict_one(x, self.root) for x in X])

In [9]:
#Supervised: Random Forest (bagging of our trees)
class RandomForestClassifierSimple:
    """
    Simple Random Forest using bootstrap samples and a subset of features at each split.
    We'll keep it minimal: bagging our DecisionTreeClassifierSimple but with feature subsampling
    implemented by passing a subset of features to the tree's fit.
    """
    def __init__(self, n_estimators=10, max_depth=5, min_samples_split=2, max_features='sqrt', random_state=0):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.rng = np.random.RandomState(random_state)
        self.trees = []
        self.features_list = []

    def _max_features_count(self, p):
        if self.max_features == 'sqrt':
            return max(1, int(np.sqrt(p)))
        elif self.max_features == 'log2':
            return max(1, int(np.log2(p)))
        elif isinstance(self.max_features, int):
            return self.max_features
        else:
            return p

    def fit(self, X, y):
        n, p = X.shape
        self.trees = []
        self.features_list = []
        m = self._max_features_count(p)
        for _ in range(self.n_estimators):
            idxs = self.rng.choice(n, n, replace=True)
            feat_idxs = self.rng.choice(p, m, replace=False)
            tree = DecisionTreeClassifierSimple(max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            tree.fit(X[idxs][:, feat_idxs], y[idxs])
            self.trees.append(tree)
            self.features_list.append(feat_idxs)
        return self

    def predict(self, X):
        preds = []
        for tree, feats in zip(self.trees, self.features_list):
            preds.append(tree.predict(X[:, feats]))
        preds = np.array(preds)
        # majority vote along axis 0
        maj = [Counter(col).most_common(1)[0][0] for col in preds.T]
        return np.array(maj)

In [10]:
#Supervised: Linear SVM (primal)

class LinearSVM_SGD:
    """
    Linear SVM via SGD on the primal hinge-loss objective with L2 regularization.
    Minimizes: (1/2)||w||^2 + C * sum(max(0, 1 - y*(w^T x)))
    where y in {-1, +1}.
    """
    def __init__(self, lr=1e-3, n_iters=1000, C=1.0, fit_intercept=True):
        self.lr = lr
        self.n_iters = n_iters
        self.C = C
        self.fit_intercept = fit_intercept

    def fit(self, X, y):
        y = np.where(y<=0, -1, 1)
        X0 = np.hstack([np.ones((X.shape[0], 1)), X]) if self.fit_intercept else X
        n, p = X0.shape
        w = np.zeros(p)
        for it in range(self.n_iters):
            margins = y * (X0.dot(w))
            # gradient from hinge loss
            mask = margins < 1
            grad = w - self.C * X0.T.dot(y * mask)
            w -= self.lr * grad
        if self.fit_intercept:
            self.intercept_ = w[0]
            self.coef_ = w[1:]
        else:
            self.intercept_ = 0.0
            self.coef_ = w
        return self

    def decision_function(self, X):
        return X.dot(self.coef_) + self.intercept_

    def predict(self, X):
        return (self.decision_function(X) >= 0).astype(int)

In [11]:
#Supervised: Gaussian Naive Bayes

class GaussianNB:
    """
    Gaussian Naive Bayes from scratch: model each feature per-class as Gaussian with estimated mean & var.
    """
    def fit(self, X, y):
        self.classes_ = np.unique(y)
        self.means_ = {}
        self.vars_ = {}
        self.priors_ = {}
        for c in self.classes_:
            Xc = X[y==c]
            self.means_[c] = Xc.mean(axis=0)
            self.vars_[c] = Xc.var(axis=0) + 1e-9
            self.priors_[c] = Xc.shape[0] / X.shape[0]
        return self

    def _log_gaussian(self, x, mean, var):
        return -0.5 * np.sum(np.log(2*np.pi*var)) -0.5 * np.sum(((x-mean)**2) / var)

    def predict(self, X):
        preds = []
        for x in X:
            best_c, best_logp = None, -np.inf
            for c in self.classes_:
                logp = np.log(self.priors_[c]) + self._log_gaussian(x, self.means_[c], self.vars_[c])
                if logp > best_logp:
                    best_logp = logp
                    best_c = c
            preds.append(best_c)
        return np.array(preds)

In [12]:
# Unsupervised: k-Means

class KMeans:
    """
    Basic k-means clustering using Lloyd's algorithm.
    """
    def __init__(self, n_clusters=3, n_iters=100, random_state=0):
        self.n_clusters = n_clusters
        self.n_iters = n_iters
        self.rng = np.random.RandomState(random_state)

    def fit(self, X):
        n, d = X.shape
        centers = X[self.rng.choice(n, self.n_clusters, replace=False)]
        for _ in range(self.n_iters):
            dists = np.linalg.norm(X[:, None, :] - centers[None, :, :], axis=2)
            labels = dists.argmin(axis=1)
            new_centers = np.array([X[labels==k].mean(axis=0) if np.any(labels==k) else centers[k] for k in range(self.n_clusters)])
            if np.allclose(new_centers, centers):
                break
            centers = new_centers
        self.cluster_centers_ = centers
        self.labels_ = labels
        return self

In [13]:
# Unsupervised: Principal Component Analysis (PCA)

class PCA:
    """
    Principal Component Analysis via SVD.
    Returns components and explained variance.
    """
    def __init__(self, n_components=None):
        self.n_components = n_components

    def fit(self, X):
        Xc = X - X.mean(axis=0)
        U, S, Vt = np.linalg.svd(Xc, full_matrices=False)
        components = Vt
        explained_variance = (S**2) / (X.shape[0]-1)
        if self.n_components is not None:
            components = components[:self.n_components]
            explained_variance = explained_variance[:self.n_components]
        self.components_ = components
        self.explained_variance_ = explained_variance
        self.mean_ = X.mean(axis=0)
        return self

    def transform(self, X):
        Xc = X - self.mean_
        return Xc.dot(self.components_.T)

In [14]:
# Unsupervised: Gaussian Mixture Model (EM)

class GaussianMixtureSimple:
    """
    Simplified Gaussian Mixture Model with full covariance and EM - The Expectation-Maximization (EM) algorithm is a common method
    for training GMMs, iteratively refining the model parameters (means, covariances, and mixing coefficients of the Gaussians) 
    to maximize the likelihood of the observed data. 
    """
    def __init__(self, n_components=2, n_iters=100, tol=1e-4, random_state=0):
        self.n_components = n_components
        self.n_iters = n_iters
        self.tol = tol
        self.rng = np.random.RandomState(random_state)

    def _multivariate_gaussian(self, X, mean, cov):
        d = X.shape[1]
        cov_inv = np.linalg.inv(cov)
        denom = np.sqrt(((2*np.pi)**d) * np.linalg.det(cov))
        diffs = X - mean
        exponents = np.einsum('ij,jk,ik->i', diffs, cov_inv, diffs)
        return np.exp(-0.5*exponents) / (denom + 1e-12)

    def fit(self, X):
        n, d = X.shape
        # initialize
        means = X[self.rng.choice(n, self.n_components, replace=False)]
        covs = np.array([np.cov(X, rowvar=False) + 1e-6*np.eye(d) for _ in range(self.n_components)])
        pis = np.ones(self.n_components) / self.n_components
        prev_ll = None
        for it in range(self.n_iters):
            # E-step
            resp = np.zeros((n, self.n_components))
            for k in range(self.n_components):
                resp[:, k] = pis[k] * self._multivariate_gaussian(X, means[k], covs[k])
            row_sums = resp.sum(axis=1)[:, None]
            resp = resp / (row_sums + 1e-12)
            Nk = resp.sum(axis=0)
            # M-step
            for k in range(self.n_components):
                means[k] = (resp[:, k][:, None] * X).sum(axis=0) / Nk[k]
                diff = X - means[k]
                covs[k] = (resp[:, k][:, None, None] * np.einsum('...i,...j->...ij', diff, diff)).sum(axis=0) / Nk[k] + 1e-6*np.eye(d)
                pis[k] = Nk[k] / n
            # log-likelihood
            ll = np.sum(np.log((resp * pis).sum(axis=1) + 1e-12))
            if prev_ll is not None and abs(ll - prev_ll) < self.tol:
                break
            prev_ll = ll
        self.means_ = means
        self.covariances_ = covs
        self.weights_ = pis
        self.resp_ = resp
        self.labels_ = resp.argmax(axis=1)
        return self

In [17]:
# Demonstration / quick tests

def _demo():
    print('\n--- Linear Regression (closed form) demo')
    X, y, true_w = make_regression_data(n=100, d=3, noise=0.5)
    lr = LinearRegressionClosedForm(lam=1e-3)
    lr.fit(X, y)
    print('true_w:', true_w)
    print('estimated coef:', lr.coef_)

    print('\n--- Logistic Regression demo')
    Xc, yc = make_classification_data(n=200, d=2)
    logreg = LogisticRegressionGD(lr=0.1, n_iters=2000)
    logreg.fit(Xc, yc)
    print('logreg accuracy:', (logreg.predict(Xc) == yc).mean())

    print('\n--- k-NN demo')
    knn = KNNClassifier(k=5)
    knn.fit(Xc, yc)
    print('knn accuracy:', (knn.predict(Xc) == yc).mean())

    print('\n--- Decision Tree demo')
    tree = DecisionTreeClassifierSimple(max_depth=5)
    tree.fit(Xc, yc)
    print('tree accuracy:', (tree.predict(Xc) == yc).mean())

    print('\n--- Random Forest demo')
    rf = RandomForestClassifierSimple(n_estimators=5, max_depth=5)
    rf.fit(Xc, yc)
    print('rf accuracy:', (rf.predict(Xc) == yc).mean())

    print('\n--- Linear SVM demo')
    svm = LinearSVM_SGD(lr=1e-3, n_iters=2000)
    svm.fit(Xc, yc)
    print('svm accuracy:', (svm.predict(Xc) == yc).mean())

    print('\n--- GaussianNB demo')
    gnb = GaussianNB()
    gnb.fit(Xc, yc)
    print('gnb accuracy:', (gnb.predict(Xc) == yc).mean())

    print('\n--- k-Means demo')
    Xk, yk = make_classification_data(n=150, d=2)
    km = KMeans(n_clusters=2)
    km.fit(Xk)
    print('kmeans cluster size:', np.bincount(km.labels_))

    print('\n--- PCA demo')
    pca = PCA(n_components=2)
    pca.fit(Xk)
    print('pca components shape:', pca.components_.shape)

    print('\n--- GMM demo')
    gmm = GaussianMixtureSimple(n_components=2, n_iters=50)
    gmm.fit(Xk)
    print('gmm weights:', gmm.weights_)

    print('\n--- End of demos. For Keras/TensorFlow examples see the Keras_usage_snippet in the file.')

if __name__ == '__main__':
    _demo()


--- Linear Regression (closed form) demo
true_w: [-1.30652685  1.65813068 -0.11816405]
estimated coef: [-1.25959699  1.61345348 -0.14386056]

--- Logistic Regression demo
logreg accuracy: 1.0

--- k-NN demo
knn accuracy: 1.0

--- Decision Tree demo
tree accuracy: 1.0

--- Random Forest demo
rf accuracy: 1.0

--- Linear SVM demo
svm accuracy: 1.0

--- GaussianNB demo
gnb accuracy: 1.0

--- k-Means demo
kmeans cluster size: [75 75]

--- PCA demo
pca components shape: (2, 2)

--- GMM demo
gmm weights: [0.65817303 0.34182697]

--- End of demos. For Keras/TensorFlow examples see the Keras_usage_snippet in the file.
